In [2]:
import cv2
import numpy as np
import matplotlib as plt

In [1]:
video_path = "/home/baug8178/data/DHF1K/video/{:03d}.AVI"
output_path = "/home/baug8178/data/DHF1K/boundary/{}_{}.png"

In [3]:
from skimage.segmentation import slic
from skimage import color

def Colour_Gradient(img):

    
    # Sobel Operator
    h, w, d = img.shape

    # define filters
    horizontal = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])  # s2
    vertical = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])  # s1

    # define images with 0s
    newgradientImage = np.zeros((h, w, d))

    # offset by 1
    for channel in range(d):
        for i in range(1, h - 1):
            for j in range(1, w - 1):
                horizontalGrad = (horizontal[0, 0] * img[i - 1, j - 1, channel]) + \
                             (horizontal[0, 1] * img[i - 1, j, channel]) + \
                             (horizontal[0, 2] * img[i - 1, j + 1, channel]) + \
                             (horizontal[1, 0] * img[i, j - 1, channel]) + \
                             (horizontal[1, 1] * img[i, j, channel]) + \
                             (horizontal[1, 2] * img[i, j + 1, channel]) + \
                             (horizontal[2, 0] * img[i + 1, j - 1, channel]) + \
                             (horizontal[2, 1] * img[i + 1, j, channel]) + \
                             (horizontal[2, 2] * img[i + 1, j + 1, channel])

                verticalGrad = (vertical[0, 0] * img[i - 1, j - 1, channel]) + \
                           (vertical[0, 1] * img[i - 1, j, channel]) + \
                           (vertical[0, 2] * img[i - 1, j + 1, channel]) + \
                           (vertical[1, 0] * img[i, j - 1, channel]) + \
                           (vertical[1, 1] * img[i, j, channel]) + \
                           (vertical[1, 2] * img[i, j + 1, channel]) + \
                           (vertical[2, 0] * img[i + 1, j - 1, channel]) + \
                           (vertical[2, 1] * img[i + 1, j, channel]) + \
                           (vertical[2, 2] * img[i + 1, j + 1, channel])

                # Edge Magnitude
                mag = np.sqrt(pow(horizontalGrad, 2.0) + pow(verticalGrad, 2.0))
                # Avoid underflow: clip result
                newgradientImage[i - 1, j - 1, channel] = mag

    # now add the images r g and b
    #rgb_edge = newgradientImage[:,:,0] + newgradientImage[:,:,1] + newgradientImage[:,:,2]
    #gray_edge = np.dot(newgradientImage[...,:3], [0.299, 0.587, 0.114])
    #print(newgradientImage.shape)
    return 0.2989 * newgradientImage[:,:,0] + 0.5870 * newgradientImage[:,:,1] + 0.1140 * newgradientImage[:,:,2]


In [4]:
def optFlow(frame1, frame2):
   gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

   # Get the Shi Tomasi corners to use them as initial reference points
   corners = cv2.goodFeaturesToTrack(gray1, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
   cornerColors = np.random.randint(0, 255, (corners.shape[0], 3))

   # Create a mask image for drawing purposes
   #mask = np.zeros_like(frame1)
   hsvImg = np.zeros_like(frame1)
   hsvImg[..., 1] = 255

   # Play until the user decides to stop
   while True:
       # Save the previous frame data
       previousGray = gray1
       previousCorners = corners.reshape(-1, 1, 2)
    
       # Get the next frame
       #ret , frame = cap.read()
    
       #if ret:
       # Convert the frame to gray scale
       gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
       # Calculate optical flow
       flow = cv2.calcOpticalFlowFarneback(previousGray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
       #corners, st, err = cv2.calcOpticalFlowPyrLK(previousGray, gray, previousCorners, None, **lkParameters)
        
       # Obtain the flow magnitude and direction angle
       mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
       # Update the color image
       hsvImg[..., 0] = 0.5 * ang * 180 / np.pi
       hsvImg[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
       rgbImg = cv2.cvtColor(hsvImg, cv2.COLOR_HSV2BGR)
          
       break
        


   # When everything is done, release the capture and close all windows
   #cap.release()
   
   return rgbImg


def boundary_map(frame_prev, frame_curr, bound_prev = None):
    img = frame_prev
    labels1 = slic(img, compactness=30, n_segments=1000)
    out1  = color.label2rgb(labels1, img, kind='avg')
    gray_edge_color = Colour_Gradient(out1)
    
    gray_edge_color_temp = Colour_Gradient(optFlow(frame_prev, frame_curr))
    mean_avg = int(np.mean(gray_edge_color_temp)/2)
    new_gray_color = np.zeros((gray_edge_color_temp.shape))
    
    mean_bound = 0
    if bound_prev is not None:
       mean_bound = np.mean(bound_prev)
    
    for i in range(0, frame_curr.shape[0]):
        for j in range(0, frame_curr.shape[1]):
              if (gray_edge_color_temp[i][j] > mean_avg):
                  new_gray_color[i][j] = gray_edge_color_temp[i][j]
              if (bound_prev is not None and bound_prev[i][j] < int(mean_bound)):
                  bound_prev[i][j] = 0
                 
    if bound_prev is None:
        boundary_map = (gray_edge_color  )   *  (1 - np.exp(-0.75 * new_gray_color)) 
    else:
        boundary_map = ( (gray_edge_color  )   *  (1 - np.exp(-0.75 * new_gray_color)))  + (0.2 * bound_prev)
        
    return boundary_map

In [ ]:
def process_file(video_file):
    cap = cv2.VideoCapture(video_file)
    ret, prev_frame = cap.read()
    bound_prev = None
    frame_count = 2
    
    while(True):
        ret, frame = cap.read()
        if(ret == False):
            break
            
        if file_no == 49 and frame_count < 405:
          frame_count = frame_count + 1
        else:
          labels1 = slic(frame, compactness=30, n_segments=1000)
          out1  = color.label2rgb(labels1, frame, kind='avg')
          gray_edge = Colour_Gradient(out1)
          bound_prev = boundary_map(prev_frame, frame, bound_prev)
          output_file = output_path.format(file_no,frame_count )
          print("writing {}".format(output_file))
          cv2.imwrite(output_file, bound_prev)
          prev_frame = frame
          frame_count = frame_count + 1
    
    cap.release()
    return

for file_no in range(49,1000):
    
    video_file = video_path.format(file_no)
    
    process_file(video_file)
        
cv2.destroyAllWindows()

writing /home/baug8178/data/DHF1K/boundary/49_405.png
writing /home/baug8178/data/DHF1K/boundary/49_406.png
writing /home/baug8178/data/DHF1K/boundary/49_407.png
writing /home/baug8178/data/DHF1K/boundary/49_408.png
writing /home/baug8178/data/DHF1K/boundary/49_409.png
writing /home/baug8178/data/DHF1K/boundary/49_410.png
writing /home/baug8178/data/DHF1K/boundary/49_411.png
writing /home/baug8178/data/DHF1K/boundary/49_412.png
writing /home/baug8178/data/DHF1K/boundary/49_413.png
writing /home/baug8178/data/DHF1K/boundary/49_414.png
writing /home/baug8178/data/DHF1K/boundary/49_415.png
writing /home/baug8178/data/DHF1K/boundary/49_416.png
writing /home/baug8178/data/DHF1K/boundary/49_417.png
writing /home/baug8178/data/DHF1K/boundary/49_418.png
writing /home/baug8178/data/DHF1K/boundary/49_419.png
writing /home/baug8178/data/DHF1K/boundary/49_420.png
writing /home/baug8178/data/DHF1K/boundary/49_421.png
writing /home/baug8178/data/DHF1K/boundary/49_422.png
writing /home/baug8178/data/

In [ ]:
for file_no in range(25,1000):
    
    video_file = video_path.format(file_no)
    cap = cv2.VideoCapture(video_file)
    ret, prev_frame = cap.read()
    bound_prev = None
    frame_count = 2
    
    while(ret == True):
        ret, frame = cap.read()
        labels1 = slic(frame, compactness=30, n_segments=1000)
        out1  = color.label2rgb(labels1, frame, kind='avg')
        gray_edge = Colour_Gradient(out1)
        bound_prev = boundary_map(prev_frame, frame, bound_prev)
        output_file = output_path.format(file_no,frame_count )
        print("writing {}".format(output_file))
        cv2.imwrite(output_file, bound_prev)
        prev_frame = frame
        frame_count = frame_count + 1
        
    cap.release()
        
cv2.destroyAllWindows()